# Pulse sequence evaluation

Using arbitrary units. Energy is normalized to the standard deviation in chemical shift strengths. Reduced Planck's constant $\hbar \equiv 1$.

In [ ]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt

## Identify primitives

In [ ]:
delay = 1e-2  # time is relative to chemical shift strength
pulse_width = 5e-3
N = 3  # number of spins

In [ ]:
def get_Hsys(dipolar_strength=1e-2):
    chemical_shifts = 2*np.pi * np.random.normal(scale=1, size=(N,))
    Hcs = sum(
        [qt.tensor(
            [qt.identity(2)]*i
            + [chemical_shifts[i] * qt.sigmaz()]
            + [qt.identity(2)]*(N-i-1)
        ) for i in range(N)]
    )
    # dipolar interactions
    dipolar_matrix = 2*np.pi * np.random.normal(scale=dipolar_strength, size=(N, N))
    Hdip = sum([
        dipolar_matrix[i, j] * (
            2 * qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmaz()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmaz()]
                + [qt.identity(2)]*(N-j-1)
            )
            - qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmax()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmax()]
                + [qt.identity(2)]*(N-j-1)
            )
            - qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmay()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmay()]
                + [qt.identity(2)]*(N-j-1)
            )
        )
        for i in range(N) for j in range(i+1, N)
    ])
    return Hcs + Hdip

In [ ]:
X = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jx(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)
Y = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jy(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)
Z = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jz(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)

In [ ]:
def get_pulses(Hsys, X, Y, Z, pulse_width, delay, rot_error=0):
    rot = np.random.normal(scale=rot_error)
    pulses = [
        qt.propagator(Hsys, pulse_width),
        qt.propagator(X * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-X * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(Y * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-Y * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(Z * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-Z * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
    ]
    delay_propagator = qt.propagator(Hsys, delay)
    pulses = [delay_propagator * i for i in pulses]
    return pulses

In [ ]:
Hsys_ensemble = [get_Hsys() for _ in range(3)]
pulses_ensemble = [
    get_pulses(H, X, Y, Z, pulse_width, delay, rot_error=0.01) for H in Hsys_ensemble
]

In [ ]:
pulse_names = [
    'd', 'x', '-x', 'y', '-y', 'z', '-z'
]

In [ ]:
pulses_dict_ensemble = [
    dict(zip(pulse_names, pulses)) for pulses in pulses_ensemble
]

In [ ]:
Utarget = qt.identity(Hsys_ensemble[0].dims[0])

## Define and evaluate pulse sequences



In [ ]:
def get_fidelity(pulse_sequence, Utarget, pulses):
    pulse_list = pulse_sequence.replace(' ', '').split(',')
    Uexp = qt.identity(Utarget.dims[0])
    for p in pulse_list:
        Uexp = pulses[p] * Uexp
    return qt.metrics.average_gate_fidelity(Uexp, Utarget)

In [ ]:
def get_mean_fidelity(pulse_sequence, Utarget, pulses_dict_ensemble):
    fidelity = 0
    for pulses in pulses_dict_ensemble:
        fidelity += get_fidelity(pulse_sequence, Utarget, pulses)
    return fidelity / len(pulses_dict_ensemble)

In [ ]:
ideal6 = 'y,x,x,y,-x,-x'
yxx24 = '-y,x,-x,y,-x,-x,y,-x,x,-y,x,x,y,-x,x,-y,x,x,-y,x,-x,y,-x,-x'
# yxx24 = format_pulse_sequence(yxx24)
yxx48 = ('y,-x,-x,y,-x,-x,-y,x,x,y,-x,-x,-y,x,x,-y,x,x,y,'
         + '-x,-x,y,-x,-x,-y,x,x,y,-x,-x,-y,x,x,-y,x,x,y,'
         + '-x,-x,-y,x,x,y,-x,-x,-y,x,x')
# yxx48 = format_pulse_sequence(yxx48)

# brute-force search
bf6 = 'x,x,y,x,x,y'  # y, -z, -x, -y, x, z
bf12 = 'x,x,-y,x,x,-y,-x,-x,-y,-x,-x,-y'
bfr12 = 'x,-y,-y,x,-y,-y,x,y,y,x,y,y'

# random search
rand12 = 'x,-y,-y,x,y,x,y,x,y,x,-y,-y'

In [ ]:
get_mean_fidelity(ideal6, Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(yxx24, Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(yxx48, Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(((bf12 + ',')*4)[:-1], Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(((bfr12 + ',')*4)[:-1], Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(rand12, Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(','.join(['-y,x,-y,x,-y,x']*4), Utarget, pulses_dict_ensemble)

In [ ]:
fids = []
Uexp = qt.identity(Utarget.dims[0])
for p in ((bfr12 + ',')*4)[:-1].split(','):
    Uexp = pulses_dict_ensemble[1][p] * Uexp
    fids.append(qt.metrics.average_gate_fidelity(Uexp, Utarget))

In [ ]:
plt.plot(-np.log10(1-np.array(fids)))
plt.ylabel('Reward')
plt.xlabel('Pulse number')

In [ ]:
fids = []
Uexp = qt.identity(Utarget.dims[0])
for p in ((yxx48 + ',')*1)[:-1].split(','):
    Uexp = pulses_dict_ensemble[1][p] * Uexp
    fids.append(qt.metrics.average_gate_fidelity(Uexp, Utarget))

In [ ]:
plt.plot(-np.log10(1-np.array(fids)))
plt.ylabel('Reward')
plt.xlabel('Pulse number')